In [8]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

import torch
from torch import nn

from transformers import BertTokenizer
from transformers import BertModel
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from catalyst import dl
from catalyst import dl, utils

import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
from util import *
import joblib
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")

In [3]:
subset = list(set(all_tiers_100)-set(["PersonalizedProduct"]))

In [4]:
training_set['labels']=training_set[subset].astype(int).values.tolist()
testing_set['labels']=testing_set[subset].astype(int).values.tolist()

In [5]:
cpc_embeddings = np.fromfile("/home/martin/patentmark/cpc.emb.verse.32d.bin", dtype=np.float32).reshape((-1,32))
cpc_labelizer = joblib.load('./node2id.joblib')
cpc_lookup = {c: n for n, c in enumerate(cpc_labelizer.classes_)}

@f.collecting
def convert_cpc_codes(codes):
    for code in codes:
        if code in cpc_lookup:
            yield cpc_lookup[code]
    
def embed_cpc_codes(codes):
    embedding = np.zeros(32)
    converted = convert_cpc_codes(codes)
    
    if not converted:
        return embedding
    
    for code_id in converted:
        embedding = embedding + cpc_embeddings[code_id]
        
    return embedding / len(converted)

training_set['embedded_cpc'] = training_set.cpc_codes.apply(embed_cpc_codes)
testing_set['embedded_cpc'] = testing_set.cpc_codes.apply(embed_cpc_codes)

In [6]:
MAX_LEN_CLAIMS = 512
MAX_LEN_ABSTRACT = 160
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 100
LEARNING_RATE = 5e-5
SEED = 17
PRED_THRES = 0.4
ACCUM_STEPS = 4

In [7]:
#model_name = "allenai/longformer-base-4096"
#model_name = "albert-base-v2"
model_name = "/home/martin/IdeaProjects/phenetics/bertForPatents/"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
training_set['citing'] = training_set[['citations', 'cited_by']].apply(
        lambda row: list(set(row['citations']+row['cited_by'])), axis=1)

In [10]:
testing_set['citing'] = testing_set[['citations', 'cited_by']].apply(
        lambda row: list(set(row['citations']+row['cited_by'])), axis=1)

In [11]:
training_set['people'] = training_set[['assignees', 'inventors']].apply(lambda row: list(set(row['assignees']+row['inventors'])), axis=1)

In [12]:
testing_set['people'] = testing_set[['assignees', 'inventors']].apply(lambda row: list(set(row['assignees']+row['inventors'])), axis=1)

In [13]:
def format(t):
    CORP_TYPES = set(
        [
            "INC",
            "LLC" "CORP",
            "KK",
            "SA",
            "SRL",
            "LTD",
            "NL",
            "PTY",
            "AG",
            "GMBH",
            "KG",
            "OG",
            "LIMITED",
            "SARL",
            "BM",
            "PLC",
            "LP",
            "IP",
            "DBA",
            "CORP",
            "CO",
        ]
    )

    tokenized = strip_non_alphanum(strip_punctuation(t)).upper().split(" ")
    cleaned = [t for t in tokenized if t not in CORP_TYPES]
    return "".join(cleaned)


people_coder = CountVectorizer(analyzer=lambda x: map(format, x), min_df=2)

In [14]:
citing_coder = CountVectorizer(analyzer=lambda x: x, min_df=4)

In [15]:
citing_coder.fit(training_set.citing)

CountVectorizer(analyzer=<function <lambda> at 0x7f93b258e940>, min_df=4)

In [16]:
people_coder.fit(training_set.people)

CountVectorizer(analyzer=<function <lambda> at 0x7f93b258e310>, min_df=2)

In [17]:
training_set['people_vec'] = list(np.array(people_coder.transform(training_set.people).todense()))
testing_set['people_vec'] = list(np.array(people_coder.transform(testing_set.people).todense()))
training_set['citing_vec'] = list(np.array(citing_coder.transform(training_set.citing).todense()))
testing_set['citing_vec'] = list(np.array(citing_coder.transform(testing_set.citing).todense()))

In [18]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer):
        self.tokenizer = tokenizer
        self.data = dataframe
        
        self.claims = dataframe.claims
        self.abstracts = dataframe.abstract
        
        self.targets = self.data.labels

    def __len__(self):
        return len(self.data)

    def tokenize(self, text, max_len, prefix=""):
        text = str(text)
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            f"input_ids": torch.tensor(ids, dtype=torch.long),
            f"attention_mask": torch.tensor(mask, dtype=torch.long),
            f"token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        }
    
            
    def __getitem__(self, index):
        
        abstract = self.tokenize(self.abstracts[index], max_len=MAX_LEN_ABSTRACT, prefix="abstract_")
        claims = self.tokenize(self.claims[index], MAX_LEN_CLAIMS, prefix="claims_")
        
        #cpcs = torch.tensor(np.array(self.data.cpc_vec[index].values), dtype=torch.float)
        people = torch.tensor(np.array(self.data.people_vec[index]), dtype=torch.float)
        citing = torch.tensor(np.array(self.data.citing_vec[index]), dtype=torch.float)
        embedded_cpc = torch.tensor(np.array(self.data.embedded_cpc[index]), dtype=torch.float)
        return {"abstract": abstract, 
                "claims": claims, 
                
                #'cpcs': cpcs,
                 'people': people,
                 'citing': citing,
                 'embedded_cpc': embedded_cpc,
                 'targets': torch.tensor(self.targets[index], dtype=torch.float)}

In [19]:
training_dataset = MultiLabelDataset(training_set, tokenizer)
testing_dataset = MultiLabelDataset(testing_set, tokenizer)

In [20]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_dataset, **train_params)
testing_loader = DataLoader(testing_dataset, **test_params)

In [21]:
loaders = {"train": training_loader, "valid": testing_loader}

In [22]:
base_model = AutoModel.from_pretrained(model_name, gradient_checkpointing=True)

In [23]:
import torch.nn.functional as F

class Model2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.text_embedder = AutoModel.from_pretrained(model_name, gradient_checkpointing=True)
        
        for param in self.text_embedder.parameters():
            param.requires_grad = False
        #self.cpc_embedder = torch.nn.EmbeddingBag.from_pretrained(torch.from_numpy(cpc_embeddings))
        
        self.people_embedder = torch.nn.Linear(len(people_coder.vocabulary_), 16)
        self.citing_embedder = torch.nn.Linear(len(citing_coder.vocabulary_), 16)
        
        total_embedding_size = self.text_embedder.pooler.dense.out_features*2+32+16*2
        output_size = len(subset)
        bottleneck = 768
        
        #self.batch_norm1 = nn.BatchNorm1d(total_embedding_size)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.Linear(total_embedding_size, bottleneck)
        
        #self.batch_norm2 = nn.BatchNorm1d(bottleneck)
        #self.dropout2 = nn.Dropout(0.1)
        #self.dense2 = nn.utils.weight_norm(nn.Linear(bottleneck, output_size))
        self.classifier = nn.Linear(bottleneck, output_size)

            
    def forward(self, abstract, claims, embedded_cpc, people, citing):
        
        abstract_emb = self.text_embedder(input_ids=abstract["input_ids"], attention_mask=abstract["attention_mask"])
        abstract_emb = abstract_emb[0][:, 0]
        
        claim_emb = self.text_embedder(input_ids=claims["input_ids"], attention_mask=claims["attention_mask"])
        claim_emb = claim_emb[0][:, 0]
        
        people_emb = F.elu(self.people_embedder(people))
        citing_emb = F.elu(self.citing_embedder(citing))
    
        x = torch.cat((abstract_emb, claim_emb, embedded_cpc, people_emb, citing_emb), 1)
        #x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.elu(self.dense1(x))
        
        #x = self.batch_norm2(x)
        #x = self.dropout2(x)
        x = self.classifier(x)
        
        return x

model = Model2()

In [24]:
device = utils.get_device()

In [25]:
from datetime import datetime
logdir="logdir/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

In [26]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir logdir/fit/ --bind_all

In [28]:
import catalyst.contrib as contrib

In [29]:
criterion = torch.nn.BCEWithLogitsLoss()
#criterion = contrib.nn.criterion.LovaszLossMultiLabel()
optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LEARNING_RATE)
#scheduler = contrib.nn.OneCycleLRWithWarmup(optimizer, num_steps=500, lr_range=(1e-4, 1e-8), init_lr=1e-9, warmup_fraction=0.2)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
#lrfinder = dl.LRFinder(final_lr=1)

runner = dl.SupervisedRunner(input_key=("abstract", "claims", "embedded_cpc", "people", "citing"))
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir=logdir,
    num_epochs=EPOCHS,
    callbacks=[
               dl.MultiLabelAccuracyCallback(threshold=PRED_THRES, activation="None"),
               dl.EarlyStoppingCallback(patience=3, metric="multi_label_accuracy", minimize=False),
               dl.TensorboardLogger(),
               #dl.CheckpointCallback(),
               dl.OptimizerCallback(accumulation_steps=ACCUM_STEPS),
               dl.ValidationManagerCallback(),
               ],
               #dl.MetricManagerCallback(num_classes=len(subset), )],
    
    fp16=True,
    verbose=True
)

1/100 * Epoch (train):   0% 0/122 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1/100 * Epoch (train): 100% 122/122 [01:40<00:00,  1.22it/s, loss=0.481, multi_label_accuracy=0.784]
1/100 * Epoch (valid): 100% 31/31 [00:25<00:00,  1.23it/s, loss=0.434, multi_label_accuracy=0.803]
[2020-12-02 19:41:19,451] 
1/100 * Epoch 1 (_base): lr=5.000e-05 | momentum=0.9000
1/100 * Epoch 1 (train): loss=0.5654 | multi_label_accuracy=0.7225
1/100 * Epoch 1 (valid): loss=0.5328 | multi_label_accuracy=0.7372
2/100 * Epoch (train): 100% 122/122 [01:43<00:00,  1.18it/s, loss=0.616, multi_label_accuracy=0.682]
2/100 * Epoch (valid): 100% 31/31 [00:24<00:00,  1.25it/s, loss=0.456, multi_label_accuracy=0.803]
[2020-12-02 19:43:34,517] 
2/100 * Epoch 2 (_base): lr=5.000e-05 | momentum=0.9000
2/100 * Epoch 2 (train): loss=0.5325 | multi_label_accuracy=0.7292
2/100 * Epoch 2 (valid): loss=0.5287 | multi_label_accuracy=0.7299
3/100 * Epoch (train): 100% 122/122 [01:42<00:00,  1.19it/s, loss=0.457, multi_label_accuracy=0.739]
3/100 * Epoch (valid): 100% 31/31 [00:24<00:00,  1.25it/s, loss=0

In [ ]:
logdir

In [ ]:
predictions = np.vstack(list(map(
    lambda x: x["logits"].cpu().numpy(), 
    runner.predict_loader(loader=loaders["valid"], resume=f"{logdir}/checkpoints/best.pth" )
)))

In [ ]:
binary_predictions = torch.sigmoid(torch.from_numpy(predictions)) > 0.5

In [ ]:
from sklearn.metrics import *
print(classification_report(testing_set[subset].astype(int), binary_predictions, target_names=subset))

In [ ]:
precision    recall  f1-score   support

     SpecificationofUse_JointReplacement       0.21      0.32      0.25        44
                                 Imaging       0.55      1.00      0.71       133
                          SurgicalMethod       0.00      0.00      0.00        40
     Manufacturing_AdditiveManufacturing       0.00      0.00      0.00        38
                      Imaging_Ultrasound       0.00      0.00      0.00        32
                             Imaging_MRI       0.34      0.20      0.26        59
AnatomicalTarget_UpperExtremity_Shoulder       0.00      0.00      0.00        23
              SpecificationofUse_Disease       0.00      0.00      0.00        30
             PersonalizedProduct_Implant       0.51      1.00      0.68       124
                           Manufacturing       0.34      0.90      0.49        83
         AnatomicalTarget_UpperExtremity       0.00      0.00      0.00        31
                     AnalysisAndModeling       0.36      0.96      0.52        84
         AnatomicalTarget_LowerExtremity       0.47      1.00      0.63       113
                      SpecificationofUse       0.34      0.99      0.50        79
                        AnatomicalTarget       0.67      1.00      0.81       164
           PersonalizedProduct_Guide/Jig       0.49      1.00      0.66       120
            AnatomicalTarget_Torso_Spine       0.00      0.00      0.00        21
                              Imaging_CT       0.29      0.31      0.30        59
          AnalysisAndModeling_3DModeling       0.30      0.93      0.46        71
    AnatomicalTarget_LowerExtremity_Knee       0.34      0.78      0.48        82
                  AnatomicalTarget_Torso       0.00      0.00      0.00        35
     AnatomicalTarget_LowerExtremity_Hip       0.00      0.00      0.00        40

                               micro avg       0.43      0.71      0.54      1505
                               macro avg       0.24      0.47      0.31      1505
                            weighted avg       0.36      0.71      0.47      1505
                             samples avg       0.43      0.74      0.52      1505

In [ ]:
# Longformer base (claims + abstract)
                                            precision    recall  f1-score   support

                     AnalysisAndModeling       0.35      1.00      0.51        84
                        AnatomicalTarget       0.67      1.00      0.81       164
            AnatomicalTarget_Torso_Spine       0.00      0.00      0.00        21
     AnatomicalTarget_LowerExtremity_Hip       0.00      0.00      0.00        40
                             Imaging_MRI       0.00      0.00      0.00        59
                                 Imaging       0.55      1.00      0.71       133
                           Manufacturing       0.34      0.99      0.50        83
             PersonalizedProduct_Implant       0.51      1.00      0.68       124
              SpecificationofUse_Disease       0.00      0.00      0.00        30
                      SpecificationofUse       0.34      0.89      0.49        79
     SpecificationofUse_JointReplacement       0.00      0.00      0.00        44
                  AnatomicalTarget_Torso       0.00      0.00      0.00        35
         AnatomicalTarget_UpperExtremity       0.00      0.00      0.00        31
                      Imaging_Ultrasound       0.00      0.00      0.00        32
                              Imaging_CT       0.32      0.25      0.28        59
          AnalysisAndModeling_3DModeling       0.28      0.80      0.42        71
                          SurgicalMethod       0.00      0.00      0.00        40
AnatomicalTarget_UpperExtremity_Shoulder       0.00      0.00      0.00        23
    AnatomicalTarget_LowerExtremity_Knee       0.34      1.00      0.51        82
           PersonalizedProduct_Guide/Jig       0.49      1.00      0.66       120
         AnatomicalTarget_LowerExtremity       0.47      1.00      0.63       113
     Manufacturing_AdditiveManufacturing       0.00      0.00      0.00        38

                               micro avg       0.44      0.69      0.54      1505
                               macro avg       0.21      0.45      0.28      1505
                            weighted avg       0.34      0.69      0.45      1505
                             samples avg       0.44      0.73      0.52      1505

In [ ]:
    #Albert base w/ 256 length sequences (claims + abstract)                
    
    precision    recall  f1-score   support

         AnatomicalTarget_LowerExtremity       0.47      1.00      0.63       113
     Manufacturing_AdditiveManufacturing       0.67      0.05      0.10        38
                                 Imaging       0.55      1.00      0.71       133
                          SurgicalMethod       0.00      0.00      0.00        40
AnatomicalTarget_UpperExtremity_Shoulder       0.18      0.13      0.15        23
              SpecificationofUse_Disease       0.00      0.00      0.00        30
    AnatomicalTarget_LowerExtremity_Knee       0.45      0.40      0.43        82
                      SpecificationofUse       0.35      0.95      0.52        79
         AnatomicalTarget_UpperExtremity       0.00      0.00      0.00        31
            AnatomicalTarget_Torso_Spine       0.00      0.00      0.00        21
             PersonalizedProduct_Implant       0.51      1.00      0.68       124
                     AnalysisAndModeling       0.38      0.65      0.48        84
          AnalysisAndModeling_3DModeling       0.33      0.68      0.44        71
                  AnatomicalTarget_Torso       0.00      0.00      0.00        35
     SpecificationofUse_JointReplacement       0.18      0.68      0.28        44
                        AnatomicalTarget       0.67      1.00      0.81       164
                           Manufacturing       0.32      0.87      0.47        83
                             Imaging_MRI       0.26      0.15      0.19        59
                      Imaging_Ultrasound       0.20      0.12      0.15        32
                              Imaging_CT       0.32      0.34      0.33        59
     AnatomicalTarget_LowerExtremity_Hip       0.14      0.03      0.04        40
           PersonalizedProduct_Guide/Jig       0.50      1.00      0.66       120

                               micro avg       0.43      0.67      0.52      1505
                               macro avg       0.29      0.46      0.32      1505
                            weighted avg       0.39      0.67      0.47      1505
                             samples avg       0.43      0.70      0.51      1505

In [ ]:
hamming_loss(testing_set[subset], binary_predictions)